In [2]:
import re
from tqdm import tqdm
import pandas as pd

In [3]:
with open('big.txt', 'r') as fd:
    lines = fd.readlines()
    words = []
    for l in lines:
        words+= re.findall('\w+',l.lower())

print("Total words in big txt : ",len(words))
vocab = list(set(words))
print("Total unique words : ",len(vocab))

Total words in big txt :  1115585
Total unique words :  32198


In [4]:
word_prob = {}

for w in tqdm(vocab):
    word_prob[w] = float(words.count(w)/len(words))

100%|██████████| 32198/32198 [13:19<00:00, 40.28it/s]


In [6]:
def split(word):
    parts = []
    for i in range(len(word)+1):
        parts+=[(word[:i],word[i:])]
    return parts

In [7]:
def delete(word):
    output = []
    for l,r in split(word):
        output.append(l+r[1:])
    return output


In [8]:
def swap(word):
    output = []
    for l,r in split(word):
        if (len(r)>1):
            output.append(l+r[1]+r[0]+r[2:])
    return output

In [16]:
def replace(word):
    charac = 'abcdefghijklmnopqrstuvwxyz'
    output = []
    for l,r in split(word):
        for ch in charac:
            output.append(l+ch+r[1:])
    return output


In [14]:
def insert(word):
    charac = 'abcdefghijklmnopqrstuvwxyz'
    output = []
    for l,r in split(word):
        for ch in charac:
            output.append(l+ch+r)
    return output


# #predicting the word (level-1)

In [11]:
def edit(word):
    return list(set(insert(word) + delete(word) + replace(word) + swap(word)))

In [12]:
def spell_check_edit_1(word,count=5):
    output = []
    sug_word = edit(word)
    for wrd in sug_word:
        if wrd in word_prob.keys():
            output.append([wrd,word_prob[wrd]])
    return list(pd.DataFrame(output,columns=['word','prob']).sort_values(by='prob',ascending=False).head(count)['word'].values)

In [17]:
spell_check_edit_1('vishal')

['visual']

# #Predicting the word(level-2)

In [20]:
def spell_check_edit_2(word, count = 5):
    
    output = []
    suggested_words = edit(word)      
    
    for e1 in edit(word):
        suggested_words += edit(e1)    
    
    suggested_words = list(set(suggested_words))
    
    for wrd in suggested_words:
        if wrd in word_prob.keys():
            output.append([wrd, word_prob[wrd]])
    return list(pd.DataFrame(output, columns = ['word','prob']).sort_values(by = 'prob', ascending = False).head(count)['word'].values)
        
spell_check_edit_2('vishal')

['vital', 'distal', 'fiscal', 'dismal', 'vista']